In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#sensor data
#Imu sensor data
a_k_1 = np.identity(3)

#GPS data
host_pos = np.array((1,2))

In [5]:
#Prediction model equations
delta_x_k = np.array([0,0,0]).reshape(3,1)
dt = 0
R = np.zeros((3,3))

F = np.identity(9)
F[:3, 3:6] = np.identity(3)
F[3:6, 6:] = -np.dot(R,a_k_1)*dt

L = np.array([[0, 0],
             [1, 0],
             [0, 1]])


In [7]:
#Measurement model equations
y = np.array([0, 0, 0]).reshape(3,1)
H = np.array([1, 0, 0])
u = np.array([0, 0, 0]).reshape(3,1)

In [ ]:
#x_est = initialize with ground trouth
x_est = np.array([0,0,0]).reshape(3,1)
while True:
    
    delta_x_pred = x_est
    
    
    
    #propogate uncertanity
    P_k = (np.dot(F_k_1, P_k_1)).dot(F_k_1.T)  + (np.dot(L_k_1, Q_k_1)).dot(K_k_1.T)